# Fine Tuning of Open-Source LLM with Custom Dataset for Text Generation

Using a pre-trained open-source LLM, we will fine-tune the model with our own data, adjusting the LLM for a specific task: generating medical text in response to a patient's clinical description.

In [4]:
# !pip install accelerate peft bitsandbytes trl datasets

In [11]:
# Imports
import sys
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import TrainingArguments
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Verifica o modelo da GPU
if torch.cuda.is_available():
    print('GPU count:', torch.cuda.device_count())
    print('GPU model:', torch.cuda.get_device_name(0))
    print('Total GPU Memory [GB]:',torch.cuda.get_device_properties(0).total_memory / 1e9)

GPU count: 1
GPU model: NVIDIA GeForce RTX 4060 Ti
Total GPU Memory [GB]: 8.585216


In [6]:
# GPU memory reset (when needed)
torch.cuda.empty_cache()

## Load Dataset

In [7]:
# Carrega o dataset
dataset = load_dataset("nlpie/Llama2-MedTuned-Instructions")

README.md:   0%|          | 0.00/2.72k [00:00<?, ?B/s]

(…)-00000-of-00001-a8790d88efc2bc45.parquet:   0%|          | 0.00/91.1M [00:00<?, ?B/s]

(…)-00000-of-00001-b543c64b1786c03e.parquet:   0%|          | 0.00/6.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/70066 [00:00<?, ? examples/s]

In [13]:
sys.getsizeof(dataset)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 200252
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 70066
    })
})


In [14]:
# Loop to view the first 3 records
for i in range(3):
    data = dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("\n-----------------------------\n")

Data Point 1:
Instruction: In your role as a medical professional, address the user's medical questions and concerns.
Input: My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.
Output: Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.

-----------------------------



### Let's work with smaller samples (to speed ​​up model training time)

In [15]:
# Training sample
dataset["train"] = dataset["train"].select(range(3500))

# Test sample
dataset["test"] = dataset["train"].select(range(300))

## Adjusting the Input Prompt Format for LLM

In [16]:
# Função para criação do prompt
def create_prompt(sample):
    prompt = sample['instruction']
    prompt += sample['input']
    single_turn_prompt = f"Instruction: {prompt}<|end_of_turn|>AI Assistant: {sample['output']}"
    return single_turn_prompt

In [18]:
# Example of using the function
print(create_prompt(dataset["train"][0]))

Instruction: In your role as a medical professional, address the user's medical questions and concerns.My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.<|end_of_turn|>AI Assistant: Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.


## Quantization Parameters

Large Language Model (LLM) quantization is a process of reducing the precision of a large language model (LLM)'s parameters to a lower value without losing much of its performance. This is done to reduce the complexity of the model and consequently improve computational efficiency and storage efficiency.

LLM quantization is useful in several situations, such as:

- Reducing model size: Quantization can reduce the size of the model, which can improve storage and data transfer efficiency.

- Improving computational efficiency: Quantization can improve the computational efficiency of the model, making it faster and more efficient in terms of computational resources.

- Increasing scalability: Quantization can allow models to be trained on cheaper and more efficient hardware, which can be useful in production applications.

BitsAndBytesConfig is used to configure LLM quantization. It is an object that contains information about quantization, such as:

- Quantization type: the type of quantization to use, such as floating-point quantization or integer quantization.

- Number of bits: the number of bits to use to represent the model parameters.

BitsAndBytesConfig is used to configure LLM quantization and can be used in conjunction with other parameters to train a more efficient and scalable language model.

In [19]:
# Sets the quantization parameters
bnb_config = BitsAndBytesConfig(load_in_4bit = True,
                                bnb_4bit_quant_type = "nf4",
                                bnb_4bit_compute_dtype = "float16",
                                bnb_4bit_use_double_quant = True)

## Loading the LLM and Tokenizer

https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

In [ ]:
# Repository name on Hugging Face
repository_hf = "berkeley-nest/Starling-LM-7B-alpha"

# Load the LLM applying quantization
llm_model = AutoModelForCausalLM.from_pretrained(repository_hf,
                                                  quantization_config = bnb_config,
                                                  device_map = "auto",
                                                  use_cache = False)

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [21]:
# Load the LLM tokenizer
tokenizer = AutoTokenizer.from_pretrained(repository_hf)

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

In [22]:
# Sets the end-of-sentence token
tokenizer.pad_token = tokenizer.eos_token

In [23]:
# Set the padding direction
tokenizer.padding_side = "right"

## Generating Response with LLM Before Fine-Tuning

In [24]:
# Function to generate LLM response
def generates_response_before_fine_tuning(prompt, model):

    # Apply the tokenizer
    encoded_input = tokenizer(prompt,
                              return_tensors = "pt",
                              add_special_tokens = True)

    # Transform the input into a tensor
    model_inputs = encoded_input.to('cuda')

    # Generate the response
    generated_ids = model.generate(**model_inputs,
                                   max_new_tokens = 1024,
                                   do_sample = True,
                                   pad_token_id = tokenizer.eos_token_id)

    # Decode the response
    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")

In [26]:
# Prompt example
prompt = """Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. """
prompt += '''Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|>'''
prompt += "AI Assistant:"

print(prompt)

Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|>AI Assistant:


In [28]:
# Generate response
generates_response_before_fine_tuning(prompt, llm_model)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'<s> Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:\nContradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|> AI Assistant: Entailment<|end_of_turn|>'

## LoRa Parameters for PEFT

## PEFT and LORA

PEFT (Parameter-Efficient Fine-Tuning) and LoRA (Low-Rank Adaptation) are fine-tuning techniques used to train large language models more efficiently, reducing computational costs and the number of parameters required to adapt to new tasks.

### 1. **PEFT (Parameter-Efficient Fine-Tuning)**:

- **Description**: It is a technique that allows tuning large models with parameter efficiency. Instead of tuning all the model parameters during training, PEFT tunes only a small fraction of the parameters, which reduces training time and cost.
- **Advantages**:
- Requires fewer computational resources.
- Improves memory efficiency.
- Keeps most of the model frozen (unchanged), tuning only a specific part for new tasks.

### 2. **LoRA (Low-Rank Adaptation)**:

- **Description**: LoRA is a specific technique within the PEFT concept that fine-tunes models by training low-rank matrices instead of training all the model parameters. The idea is to insert small additional low-rank matrices in some of the model layers. Instead of training all the layers, only these extra matrices are trained, and the original weights remain frozen.
- **How ​​it works**: LoRA decomposes the weight matrix of a layer into two smaller (or low-rank) matrices that, when multiplied, approximate the original weights. During fine-tuning, only these low-rank matrices are trained.
- **Advantages**:
- Efficient fine-tuning of large models without directly modifying the main weights.
- Uses less memory and computing resources.
- Facilitates fine-tuning for various tasks, keeping the model base unchanged.

Both techniques are used to improve the efficiency of fine-tuning large models, such as GPT and BERT, for specific tasks, making them more accessible and feasible in resource-constrained environments.
<br/>
<br/>

Here is the description of each parameter in the LoraConfig configuration we'll use:

**r**: Specifies the size of the expansion factor used in the LoRA (Low-Rank Adaptation) mechanism. A value of r = 8 indicates that low-rank adaptation increases the dimensionality of the projections by a factor of 8. Essentially, this parameter controls the size of the dimensionality increase that is applied to selected layers of the model to allow for greater adaptability without significantly changing the total number of model parameters.

**lora_alpha**: Multiplier for the learning factor applied specifically to the LoRA adaptation parameters. With lora_alpha = 16, the learning of these specific parameters is increased by 16 times compared to the rest of the model parameters. This allows the adaptation parameters to adjust more quickly during training.

**lora_dropout**: Dropout rate applied to LoRA modifications. A value of 0.05 means that 5% of the elements in the LoRA projections will be randomly zeroed out during training. Dropout is a regularization technique used to avoid overfitting by forcing the model to learn more robust representations since it cannot rely on any single connection between neurons.

**bias**: Sets whether or not bias terms are used in LoRA fitting. Here, it is set to "none", indicating that no bias terms will be added to LoRA fitting. Choosing not to use bias can simplify the model and focus the fitting solely on the weights.

**task_type**: Sets the type of task the model is being configured for. "CAUSAL_LM" refers to a causal language model, where the prediction of each subsequent word depends only on the previous words and not on any future words. This is typical of models that generate text in a sequential manner, such as text generation or dialogue modeling.

These parameters are configured to tune the model effectively and specifically, leveraging the low-rank adaptation technique to fine-tune the model without the computational cost of re-training all the parameters of the original model.

In [29]:
# Defines LoRa parameters
peft_config = LoraConfig(r = 8,
                             lora_alpha = 16,
                             lora_dropout = 0.05,
                             bias = "none",
                             task_type = "CAUSAL_LM")

In [30]:
# Prepare the model for fine-tuning
llm_model = prepare_model_for_kbit_training(llm_model)

# Concatenate the base model with the LoRa parameters
llm_model = get_peft_model(llm_model, peft_config)

## Training Arguments

The parameters of the TrainingArguments object are used to configure the training process of a machine learning model. Here is the description of each of them:

**output_dir**: Directory where the artifacts of the trained model, such as model weights and settings, will be saved. Here, it is set to "fitted_model".

**per_device_train_batch_size**: Number of examples processed simultaneously on each device (such as a GPU) during training.

**gradient_accumulation_steps**: Number of training steps to accumulate the gradients before performing a parameter update. This allows using a larger effective batch size than what can fit in the device's memory.

**optim**: Optimizer used to update the model weights. "paged_adamw_32bit" is a variation of the AdamW optimizer that optimizes memory usage by using 32-bit precision.

**learning_rate**: Initial learning rate used by the optimizer.

**lr_scheduler_type**: Type of learning rate scheduler, which adjusts the learning rate based on the number of epochs or steps. "cosine" indicates that the learning rate follows a cosine curve, gradually decreasing over the epochs.

**save_strategy**: Strategy for saving the model during training. "epoch" means that the model will be saved at the end of each epoch.

**logging_steps**: Number of training steps after which logs will be recorded. Here, it is set to 10, meaning that the training progress will be logged every 10 steps.

**num_train_epochs**: Total number of training epochs. An epoch is a complete pass through the training data.

**max_steps**: Maximum number of training steps to be executed. Useful to terminate training after a fixed number of steps, regardless of epochs.

**fp16**: If enabled, enables the use of 16-bit floating point during training to reduce memory usage and possibly speed up computation. Here, it is set to True, indicating that it is enabled.

In [31]:
# Defines the model training arguments
training_arguments = TrainingArguments(output_dir = "adjusted_model",
                                           per_device_train_batch_size = 1,
                                           gradient_accumulation_steps = 4,
                                           optim = "paged_adamw_32bit",
                                           learning_rate = 2e-4,
                                           lr_scheduler_type = "cosine",
                                           save_strategy = "epoch",
                                           logging_steps = 10,
                                           num_train_epochs = 1,
                                           max_steps = 250,
                                           fp16 = True)

## Supervised Fine-tuning Trainer (SFFT) Parameters

The **Supervised Fine-Tuning Trainer** is a method used in the supervised training of large language models, such as GPT and BERT, to fine-tune their capabilities for a specific task. It involves updating the model's parameters based on labeled data, so that the model can learn to perform tasks such as classification, translation, or text generation more effectively.

### Structure of the Supervised Fine-Tuning Trainer:

1. **Pre-trained model**:
- It starts with a model that has already been pre-trained on a large corpus of unsupervised data. This pre-training gives the model a solid foundation in natural language, but it is not yet fine-tuned for specific tasks.
2. **Supervised data**:
- For supervised fine-tuning, a labeled dataset is used, where the inputs have expected outputs. Example: For a sentiment classification task, texts would come with labels such as "positive" or "negative".
3. **Objective**:
- During supervised training, the goal is to minimize a loss function, which measures how different the model's output is from the expected output on the supervised data.
4. **Parameter update**:
- The model adjusts its parameters based on the predictions it makes regarding the data labels. The error is calculated and the **optimizer**, such as **Adam**, adjusts the weights of the model layers to reduce this error next time.

### Workflow:
1. **Forward pass**: The input data is passed through the model to generate a prediction.
2. **Loss calculation**: The prediction is compared with the expected label and the loss function is calculated.
3. **Backpropagation**: Through gradient descent, the errors are propagated to adjust the model parameters. 4. **Parameter update**: The optimizer updates the weights to reduce the error in the next iteration.

### Tools:

The supervised fine-tuning process is usually managed by frameworks such as **Hugging Face Transformers**, where the **Trainer** is a class that facilitates this process. It takes care of many technical aspects, such as:

- Splitting the dataset into training and validation sets.
- Calculating the loss function and updating the parameters.
- Evaluating metrics of the model's performance.

### Importance:

The Supervised Fine-Tuning Trainer is crucial for customizing large models, such as GPT, for specific tasks, allowing the model to generalize well and perform better in real-world applications.

Here is a description of each parameter used to create an SFTTrainer instance:

**model**: Defines the machine learning model that will be trained.

**peft_config**: Configuration for parameter efficient model training (PEFT).

**max_seq_length**: Maximum sequence length that the model can accept. It is set to 512, which means that each input (such as text) will be truncated or padded to this length to maintain a consistent length across inputs.

**tokenizer**: The tokenizer is used to convert text into a format that the model can process (such as tokens or numbers). Here, a specific tokenizer is used, which must be compatible with the chosen model.

**packing**: A boolean parameter that, when true, indicates that the trainer should attempt to efficiently pack the input sequences to optimize memory usage and training performance.

**formatting_func**: Function used to format the input data before it is fed to the model.

**args**: Additional training configuration parameters, grouped in dsa_training_arguments. These arguments define various settings such as learning rate, saving strategy, among others, which were previously described.

**train_dataset**: Dataset used to train the model.

**eval_dataset**: Dataset used to evaluate the model.

In [32]:
# Define os parâmetros do SFTT
trainer = SFTTrainer(model = llm_model,
                         peft_config = peft_config,
                         max_seq_length = 512,
                         tokenizer = tokenizer,
                         packing = True,
                         formatting_func = create_prompt,
                         args = training_arguments,
                         train_dataset = dataset["train"],
                         eval_dataset = dataset["test"])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## LLM Training (Fine-Tuning)

In [33]:
%%time
trainer.train()

  0%|          | 0/250 [00:00<?, ?it/s]

{'loss': 2.1541, 'grad_norm': 2.9172513484954834, 'learning_rate': 0.00019936113105200085, 'epoch': 0.02}
{'loss': 1.8915, 'grad_norm': 1.6502469778060913, 'learning_rate': 0.00019745268727865774, 'epoch': 0.04}
{'loss': 1.9087, 'grad_norm': 1.7545417547225952, 'learning_rate': 0.00019470983049947444, 'epoch': 0.06}
{'loss': 1.6634, 'grad_norm': 1.9404270648956299, 'learning_rate': 0.0001899405251566371, 'epoch': 0.08}
{'loss': 1.5109, 'grad_norm': 1.562394142150879, 'learning_rate': 0.0001837528040042142, 'epoch': 0.1}
{'loss': 1.4148, 'grad_norm': 1.895841121673584, 'learning_rate': 0.0001762442511011448, 'epoch': 0.13}
{'loss': 1.5128, 'grad_norm': 6.78122615814209, 'learning_rate': 0.00016753328081210245, 'epoch': 0.15}
{'loss': 1.4127, 'grad_norm': 1.6238045692443848, 'learning_rate': 0.00015775727034222675, 'epoch': 0.17}
{'loss': 1.3461, 'grad_norm': 1.735244870185852, 'learning_rate': 0.0001470703932165333, 'epoch': 0.19}
{'loss': 1.4661, 'grad_norm': 1.481718897819519, 'learni

TrainOutput(global_step=250, training_loss=1.4845002326965333, metrics={'train_runtime': 1025.9084, 'train_samples_per_second': 0.975, 'train_steps_per_second': 0.244, 'total_flos': 2.185444196352e+16, 'train_loss': 1.4845002326965333, 'epoch': 0.5211047420531527})

In [34]:
# Save the adjusted model
trainer.save_model("adjusted_model")

In [35]:
# Unload the model and remove it from training mode
final_model = llm_model.merge_and_unload()

## Generating Text with LLM

In [36]:
# LLM Response Generation Function
def generate_response_after_fine_tuning(prompt, model):

    # Apply the tokenizer
    encoded_input = tokenizer(prompt,
                              return_tensors = "pt",
                              add_special_tokens = True)

    # Transforms the input into a tensor
    model_inputs = encoded_input.to('cuda')

    # Generate the response
    generated_ids = model.generate(**model_inputs,
                                   max_new_tokens = 512,
                                   do_sample = True,
                                   use_cache = False,
                                   pad_token_id = tokenizer.eos_token_id)

    # Decode the response
    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0]

---

### Inference 1

In [38]:
%%time
prompt = "Instruction: In your role as a medical professional, address the user's medical questions and concerns. "
prompt += "I have a white tab under my tounge that is not only painful when i touch it but bleeds as well. not sure what it is, or why I got it. Can you give me any advise? <|end_of_turn|> "
prompt += "AI Assistant:"
response = generate_response_after_fine_tuning(prompt, final_model)
print(response)

<s> Instruction: In your role as a medical professional, address the user's medical questions and concerns. I have a white tab under my tounge that is not only painful when i touch it but bleeds as well. not sure what it is, or why I got it. Can you give me any advise? <|end_of_turn|>  AI Assistant: It sounds like you may have a lesion known as an ulcer. The most common cause of an oral ulcer is trauma, such as from a sharp tooth or the edges of dentures. If this is the case, stopping the trauma should help the ulcer to heal.

If the ulcer is not due to trauma, it could be related to diet or may be viral or bacterial in origin. However, if the ulcer persists for longer than two to three weeks, it is important to check it out with your Dentist.<|end_of_turn|>
CPU times: total: 3.05 s
Wall time: 23.6 s


### Inference 2

In [40]:
%%time
prompt = "Instruction: In your capacity as a healthcare expert, offer insights and recommendations in response to users' medical inquiries. "
prompt += "I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped. <|end_of_turn|> "
prompt += "AI Assistant:"
response = generate_response_after_fine_tuning(prompt, final_model)
print(response)

<s> Instruction: In your capacity as a healthcare expert, offer insights and recommendations in response to users' medical inquiries. I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped. <|end_of_turn|>  AI Assistant: Hi and Thank you for using the Healthcare Magician. In light of your history of an anxiety and depression, I understand your desire to find effective treatment for these conditions. When you say you have tried various therapists and pills, it's important to understand what specifically you have tried in terms of psychotropic medication. The best treatment for an anxiety or depression involves a combination of psychotherapy and psychotropic medications. It's important to note that not all medications are effective for all individuals. Thus, a 'trial and error' approach is often taken when looking for the 'right' treatment for you. In addition, cognitive behavioural therapy is the primary form of psychotherapy used for anxie

### Inference 3

In [41]:
%%time
prompt = "Instruction: As a medical chatbot, your responsibility is to provide information and guidance on medical matters to users. "
prompt += "Hi sir, I am so happy with this website. First of all thanks for giving this opportunity. I am the  Software employee.My age is 24. My height is 169cm .Recently I got back pain and some pain in chest. How can i get relief from those pains.How i improve my health and which type of diseases will attack to my life in future. Please give Some health tips for heart and kidneys protection. <|end_of_turn|> "
prompt += "AI Assistant:"
response = generate_response_after_fine_tuning(prompt, final_model)
print(response)

<s> Instruction: As a medical chatbot, your responsibility is to provide information and guidance on medical matters to users. Hi sir, I am so happy with this website. First of all thanks for giving this opportunity. I am the  Software employee.My age is 24. My height is 169cm .Recently I got back pain and some pain in chest. How can i get relief from those pains.How i improve my health and which type of diseases will attack to my life in future. Please give Some health tips for heart and kidneys protection. <|end_of_turn|>  AI Assistant: Hello, I am glad to hear that you are interested in your health. Back pain and chest pain can be caused by various conditions. In order to identify the underlying cause, it is necessary to review your medical history and perform a thorough physical examination. Some of the common causes of back pain include muscle strain, disc degeneration, and arthritis. Chest pain can be caused by a variety of conditions including heart disease, apeksia nervosa, and

## The End